<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/240726_npp_analysis_cotubanama_national_park.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import TileLayer
from google.colab import files

ee.Authenticate()
ee.Initialize(project='ee-foss4g')

In [27]:
# Cotubanama National Park
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
# wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 180))
wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 768))

wdpa_name = wdpa.first().get('NAME').getInfo()
print("Name:", wdpa_name)

Name: Seoraksan


In [28]:
# 1: Pixel value for the outline, 2: Thickness of the outline (in pixels)
wdpa_raster = ee.Image().paint(wdpa, 1, 2)

# Set visualization parameters
wdpa_vis = {
    'palette': ['#FFFF00'], # Apply Yellow color
    'opacity': 1
}

m = geemap.Map(width="800px", height="400px")
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [45]:
dataset = (
    ee.ImageCollection('MODIS/061/MOD17A3HGF')
    .filterDate("2023-01-01", "2023-12-31")
    .filterBounds(wdpa)
)

npp = dataset.select('Npp')

npp_vis = {
    'min': 0,
    'max': 19000,
    'palette': ['bbe029', '0a9501', '074b03']
}

m = geemap.Map(width="800px", height="400px",
               basemap='Esri.WorldImagery')
m.addLayer(npp, npp_vis, 'NPP')
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [44]:
# Multiply the 'Npp' values by 0.0001.
scaled_npp = npp.multiply(0.0001)

# Function to set values within the range [3.2761, 3.2767] to NODATA (null).
def mask_invalid_values(image):
    mask = image.gte(3.2761).And(image.lte(3.2767)).Not()
    return image.updateMask(mask)

# Apply the function to the ImageCollection.
masked_npp = scaled_npp.map(mask_invalid_values)

# Example to print the first image in the collection (for verification).
first_image = masked_npp.first()
print(first_image.getInfo())

AttributeError: 'ImageCollection' object has no attribute 'multiply'

In [46]:
type(npp)

ee.imagecollection.ImageCollection

In [43]:
# Check the number of images in the dataset
dataset_size = dataset.size().getInfo()
print(f'The dataset contains {dataset_size} images.')

The dataset contains 1 images.


In [41]:
# Calculate the sum of Npp within the wdpa region
npp_sum = npp.sum().reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=wdpa.geometry(),
    scale=500,  # scale should be appropriate for the dataset, 500m for MODIS
    maxPixels=1e9
)

# Get the sum value and print it
npp_sum_value = npp_sum.get('Npp').getInfo()
npp_sum_value_scaled = npp_sum_value * 0.0001
print(f'The sum of NPP within the wdpa region for 2023 is: {npp_sum_value_scaled}')

The sum of NPP within the wdpa region for 2023 is: 1621.7026192156857
